In [19]:
import requests
from astropy.time import Time
import pandas as pd
import psycopg2

# User entered parameters that describe the sky area and time window for ZTF transients, when the DESI observation occurs, and the DESI magnitude cut

In [20]:
pname = "query"

ra = 220.
dec = 0.
search_radius = 10 # in degrees

nt = Time.now()
last_mjd_discovery = nt.mjd-365./4 # the earliest for the last ZTF detection
obsmjd = nt+3

mag_cut = 22

# Credentials and connection to the ALERCE database

In [21]:
params = {
        "dbname" : "ztf",
        "user" : "alerceread",
        "host": "54.205.99.47",
        "password" : "w*C*u8AXZ4e%d+zv"
}

In [22]:
conn = psycopg2.connect(dbname=params['dbname'], user=params['user'], host=params['host'], password=params['password'])

# The database query getting object with high-probability "SN" or "AGN" by Stamp Classifier

In [23]:
objects_str=f"{'{}', {}, {} }".format(pname, ra, dec)
print(objects_str)

('query', 220.0, 0.0)


In [24]:
query = """
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
),
sn (oid, classifier_name, class_name, probability, ranking) AS (
    SELECT
        o.oid, p.classifier_name, p.class_name, p.probability, p.ranking
    FROM
        probability p
    INNER JOIN 
        object o
    ON 
        o.oid=p.oid
    WHERE
        p.classifier_name='stamp_classifier'
        AND p.class_name IN ('SN','AGN')
        AND p.ranking=1
        AND p.probability > 0.4
        AND o.firstmjd > %s
)

SELECT 
    c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec, q3c_dist(c.ra,c.dec,o.meanra,o.meandec), 
    o.firstmjd, o.lastmjd, sn.classifier_name, sn.class_name, sn.probability, sn.ranking

FROM object o INNER JOIN sn ON sn.oid=o.oid, catalog c
    /*
     * It is REALLY important to first use the catalog then the object ra,dec for speed. The radius is in degrees.
     */
WHERE
    q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    AND o.firstmjd > %s
ORDER BY
    c.source_id
""" % (last_mjd_discovery, last_mjd_discovery)
query_str = query.format(values = objects_str, radius = search_radius) # radius in degrees

In [25]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches

(495, 13)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist,firstmjd,lastmjd,classifier_name,class_name,probability,ranking
0,query,220.0,0.0,ZTF19aasixog,224.192779,-2.758958,5.017734,59228.499433,59262.484225,stamp_classifier,AGN,0.660308,1
1,query,220.0,0.0,ZTF21aaisexy,215.311569,1.334834,4.874340,59233.490984,59264.412002,stamp_classifier,AGN,0.550574,1
2,query,220.0,0.0,ZTF21aalytpn,213.912667,-4.055893,7.310535,59252.524109,59264.481921,stamp_classifier,AGN,0.671337,1
3,query,220.0,0.0,ZTF21aalzcpi,220.193230,-8.538515,8.540685,59264.503518,59264.503518,stamp_classifier,AGN,0.632137,1
4,query,220.0,0.0,ZTF21aanefde,216.823055,6.030155,6.813109,59265.358854,59269.457674,stamp_classifier,AGN,0.580785,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,query,220.0,0.0,ZTF21aaogipe,215.519465,-8.331014,9.451944,59278.477164,59278.477164,stamp_classifier,SN,0.737364,1
491,query,220.0,0.0,ZTF21aapkeql,217.035917,2.126991,3.647717,59278.339722,59292.434444,stamp_classifier,SN,0.427240,1
492,query,220.0,0.0,ZTF21aapkxjm,218.114738,2.489603,3.122515,59265.420718,59290.421991,stamp_classifier,AGN,0.415869,1
493,query,220.0,0.0,ZTF21aaqclef,210.272854,-1.139439,9.793018,59292.375023,59292.375023,stamp_classifier,SN,0.784787,1


In [26]:
epoch = 2000.0     # ZTF standard is J2000
for index, da in matches.iterrows():
    response = requests.get("http://3.238.105.175:8081/parametric/sn?oid=%s&mjd=%s" % (da['oid'], obsmjd.mjd))
    forecast = response.json()["forecast"]
    if forecast[1]['magpsf'] is not None and forecast[1]['magpsf'] <= mag_cut:
        prog = 'BRIGHT' if forecast[1]['magpsf'] < 21 else 'DARK'
        print('{:<10.6f} {:>10.6f} {:>9.6f} {:>9.6f} {:>7.1f}  LW  FIBER  {:7s} {:>15.8f} {:>15.8f}'.format(
            da['meanra'], da['meandec'], 0, 0, epoch, prog, da['lastmjd'], obsmjd.mjd+14))

215.311569   1.334834  0.000000  0.000000  2000.0  LW  FIBER  DARK     59264.41200230  59323.92677610
220.193230  -8.538515  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59264.50351850  59323.92677610
216.823055   6.030155  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59269.45767360  59323.92677610
224.435599   5.758867  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59273.46185190  59323.92677610
221.695922  -5.861504  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59290.41870370  59323.92677610
226.227930   7.138013  0.000000  0.000000  2000.0  LW  FIBER  DARK     59290.42011570  59323.92677610
215.287915   3.420885  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59295.39418980  59323.92677610
213.787655  -5.243440  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59266.38136570  59323.92677610
214.182405   4.824546  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59292.43631940  59323.92677610
228.790825  -2.347770  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59271.45859

218.496452  -9.438635  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59262.48187500  59323.92677610
224.260627  -4.091274  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59262.48422450  59323.92677610
218.645610  -5.634069  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59265.35837960  59323.92677610
220.220494   6.171114  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59267.38008100  59323.92677610
217.357457   9.063352  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59297.39878470  59323.92677610
222.626407   3.864610  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59292.43444440  59323.92677610
218.993765   3.538232  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59290.42199070  59323.92677610
226.890382   3.670870  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59290.42011570  59323.92677610
228.076446  -4.736836  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59290.39583330  59323.92677610
226.111635   6.118980  0.000000  0.000000  2000.0  LW  FIBER  DARK     59292.37303

216.545380  -6.743881  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59262.48187500  59323.92677610
220.489094  -8.270103  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59262.48187500  59323.92677610
210.705655   1.190633  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59295.39418980  59323.92677610
212.077433  -2.386709  0.000000  0.000000  2000.0  LW  FIBER  DARK     59264.48192130  59323.92677610
229.436771   1.881597  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59271.45812500  59323.92677610
220.641771   9.369878  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59290.37391200  59323.92677610
225.446506  -1.246277  0.000000  0.000000  2000.0  LW  FIBER  DARK     59290.41964120  59323.92677610
227.289897   4.981781  0.000000  0.000000  2000.0  LW  FIBER  DARK     59290.42011570  59323.92677610
216.191334   8.817882  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59291.37728010  59323.92677610
225.656177   1.970419  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59292.37303

216.688786  -1.861117  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59292.37502310  59323.92677610
221.574932  -3.504722  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59292.39540510  59323.92677610
226.762495   0.738005  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59295.38995370  59323.92677610
229.476574  -0.964867  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59295.38995370  59323.92677610
221.959004  -6.222863  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59295.39231480  59323.92677610
216.825698  -5.157716  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59297.38026620  59323.92677610
222.078365  -6.140699  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59302.41939810  59323.92677610
217.232442  -6.194070  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59262.48187500  59323.92677610
223.533569   2.979507  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59271.45812500  59323.92677610
222.424582  -9.166820  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59263.47843

222.282100   5.225109  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59292.43444440  59323.92677610
219.853455   6.741241  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59292.43444440  59323.92677610
213.278055   3.747110  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59297.37979170  59323.92677610
217.908508   6.179088  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59297.38310190  59323.92677610
217.348606  -7.581968  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59262.48187500  59323.92677610
224.734945   5.072117  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59263.47891200  59323.92677610
212.187518   5.901881  0.000000  0.000000  2000.0  LW  FIBER  DARK     59264.41200230  59323.92677610
216.031974  -7.578826  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59264.50351850  59323.92677610
218.545388  -3.237053  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59271.46057870  59323.92677610
229.674280   2.210665  0.000000  0.000000  2000.0  LW  FIBER  BRIGHT   59271.45812